<a target="_blank" href="https://colab.research.google.com/github/TinArmEngineering/ltc-docs/blob/main/py-client/create-job.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Starting up
Log into the website for Tin Arm Engineering's machine solver. Under **profile**, retrieve your API key and, optionally, the organisation id that you wish to use.  

**Consider your API key like a password. Do not commit it to a repository.**

In [ ]:
pip install ltc_client pint-pandas

In [ ]:
import ltc_client
from ltc_client.api import STATUS_JOB, JOB_STATUS
import numpy as np
import pint_pandas
import time

config = {  'root_url': "https://api.ltc.tinarmengineering.com",
            'api_key': "my-api-key",
            'org_id': "my-org-id" }

api = ltc_client.Api(config["root_url"], config["api_key"], config["org_id"])

q = pint_pandas.PintType.ureg

In [ ]:
stator_parameters = {
    "slot_liner_thickness": 300 * q.um,
    "stator_bore": 8.20 * q.cm,
    "tooth_tip_depth": 1.0 * q.mm,
    "slot_opening": 1.5 * q.mm,
    "tooth_width": 9.8 * q.mm,
    "stator_outer_diameter": 0.136 * q.m,
    "back_iron_thickness": 5.5 * q.mm,
    "stator_internal_radius": 500 * q.um,
    "number_slots": 12 * q.count,
    "tooth_tip_angle": 70 * q.degrees
    }

air_gap_length = 1 * q.mm

rotor_parameters = {
    "rotor_od": stator_parameters["stator_bore"] - 2 * air_gap_length,
    "rotor_bore": 40 * q.mm,
    "banding_thickness": 0.5 * q.mm,
    "number_poles": 10 * q.count,
    "magnet_thickness": 4.5 * q.millimeter,
    "magnet_pole_arc": 150 * q.degrees,
    "magnet_inset": 0.25 * q.millimeter
    }

symmetry = np.gcd(stator_parameters["number_slots"].magnitude, rotor_parameters["number_poles"].magnitude) * q.count

winding_parameters = {
    "symmetry": symmetry,
    "number_phases": 3 * q.count,
    "number_layers": 2 * q.count,
    "coil_span": 1 * q.count,
    "turns_per_coil": 43 * q.count,
    "empty_slots": 0 * q.count,
    "fill_factor": 42 * q.percent
    }

our_machine = ltc_client.Machine(stator_parameters, rotor_parameters, winding_parameters)

simulation_parameters = {
    "samples_per_electrical_period": 45 * q.count/q.turn,
    "timestep_intervals": 45 * q.count,
    "active_length": 65 * q.mm }

op_nominal = {
    "simulated_speed": 2060 * q.rpm,
    'q_axis_current_density': 6.23 * q.A * q.mm**-2,
    "d_axis_current_density": 0 * q.A / q.mm ** 2,
    'current_angle': 255 * q.degrees
    }

job = ltc_client.Job(our_machine, op_nominal, simulation_parameters)
result = api.create_job(job)
result = api.update_job_status(job.id, JOB_STATUS['QueuedForMeshing'])

print('Job id= {id}, creation_date= {creation_date}, status= {status}'.format(**result))

while STATUS_JOB[api.get_job(job.id)['status']] != 'Complete':
    print("job 1 {0}".format(STATUS_JOB[api.get_job(job.id)['status']] ))
    time.sleep(10)

print("job 1 {0}".format(STATUS_JOB[api.get_job(job.id)['status']] ))